# Sequências - Aula Prática 02/04
## RNNs (Recurrent Neural Networks)

Neste notebook iremos continuar nossos estudos de redes neurais recorrentes (RNNs), trabalhando dessa vez com geração de nomes a nível de caractere, utilizando os mesmos dados do notebook `name_classification.ipynb`.

> Em mais detalhes, dado um *idioma*, queremos construir uma rede (*language model*) para gerar um *nome* que possui características dos nomes originários daquele idioma.

- Esse notebook foi fortemente inspirado no segundo tutorial da série [NLP From Scratch](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html), disponibilizado no site do PyTorch.

Caso esteja executando esse notebook no ambiente da Tatu, por favor execute a seguinte célula.

In [1]:
%load_ext nbproxy


Variáveis de ambiente http_proxy e https_proxy configuradas!


In [2]:
# Precisaremos instalar o pacote NLTK para conseguir realizar algumas análises
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


## Importação de pacotes

In [3]:
import torch
import random
import unicodedata

import numpy as np
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

from glob import glob
from tqdm.notebook import tqdm
from collections import defaultdict
from nltk.translate.bleu_score import sentence_bleu

In [4]:
# Verificando se temos CUDA disponível e selecionando o device que será utilizado
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device escolhido:', device)

Device escolhido: cuda


## Processamento da base de dados

Como mencionado no ínicio do notebook, iremos trabalhar com os mesmos dados vistos no notebook `name_classification.ipynb`. Porém, dessa vez, iremos trocar o que será entrada e saída do nosso modelo. No notebook anterior, tinhamos como entrada um nome (sequência) e como saída um idioma. Dessa vez, queremos admitir como entrada um idioma e *gerar* uma nome letra por letra.

Durante esse notebook, iremos trabalhar com os dados localizados na pasta `/pgeoprj/ciag2023/datasets/sequence_datasets/names/`. Nela, você verá 18 arquivos `.txt` separados por países, onde cada arquivo contém uma lista de nomes originários daquele país.

> Para esse notebook, iremos processar os dados de maneira similar, criando um dicionário que mapeia `linguagem -> lista de nomes`.

<!-- e também definindo a função `unicode2ascii` para remover caracteres especiais de idiomas específicos. -->

In [5]:
filepaths = glob('/pgeoprj/ciag2023/datasets/sequence_datasets/names/*.txt')  # * aqui quer dizer "qualquer string"
filepaths

['/pgeoprj/ciag2023/datasets/sequence_datasets/names/Arabic.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Dutch.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/English.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/French.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/German.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Greek.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Irish.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Italian.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Japanese.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Korean.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Polish.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Portuguese.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Scottish.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Spanish.txt',
 '/pgeoprj/ciag2023/datasets/sequence_datasets/names/Vietnamese.txt'

In [6]:
def get_language(filepath):
    filename = filepath.split('/')[-1]
    language = filename.split('.')[0]

    return language

print('Primeiro filepath:', filepaths[0])
print('Linguagem do primeiro filepath:', get_language(filepaths[0]))

Primeiro filepath: /pgeoprj/ciag2023/datasets/sequence_datasets/names/Arabic.txt
Linguagem do primeiro filepath: Arabic


In [7]:
languages = [] 
language_names = defaultdict(list)  # defaultdict é igual à um dict normal, porém não precisamos verificar
                                    # manualmente se uma entrada já foi inicializada, nesse caso com uma lista vazia

for filepath in filepaths:
    language = get_language(filepath)
    languages.append(language)
    
    with open(filepath, 'r') as fp:
        lines = fp.readlines()  # convertendo as linhas de um arquivo em uma lista de linhas

        for line in lines:
            line = line.strip()  # removendo \n e espaços no começo e fim da string
            language_names[language].append([*line])  # usamos * para "descompactar" string em caracteres

print('5 primeiros nomes em árabe:', language_names['Arabic'][:5])
print('Idiomas disponíveis no conjunto de dados:', languages)

5 primeiros nomes em árabe: [['K', 'h', 'o', 'u', 'r', 'y'], ['N', 'a', 'h', 'a', 's'], ['D', 'a', 'h', 'e', 'r'], ['G', 'e', 'r', 'g', 'e', 's'], ['N', 'a', 'z', 'a', 'r', 'i']]
Idiomas disponíveis no conjunto de dados: ['Arabic', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Scottish', 'Spanish', 'Vietnamese', 'Chinese', 'Czech', 'Russian']


Seguindo as ideias trabalhadas em notebooks passados, precisamos criar um mapeamento entre *tokens* e índices. No nosso caso, o *token* é uma letra do nosso alfabeto. Para isso, iremos construir nosso alfabeto (vocabulário), com base nas letras únicas de todos os nomes da base de dados, e, posteriormente, iremos criar 2 estruturas para realizar tal mapeamento: `token2index` e `index2token`.

> Além dos *tokens* obtidos através do texto, é muito comum vermos também outros 3 *tokens* especiais:
> - **\<sos\>**: abreviação para *start-of-sequence*, ou início de sentença.
> - **\<eos\>**: abreviação para *end-of-sequence*, ou fim de sentença.
> - **\<pad\>**: *token* especial destinado para indicar um valor de *pad* na nossa sequência.

- Diferentemente do notebook anterior, os *tokens* de **\<sos\>** e **\<eos\>** serão úteis, uma vez que eles são utilizados em contextos de geração de sentenças. O token **\<pad\>** ainda não será utilizado, uma vez que ainda não trabalharemos com *batches* por enquanto.
  - Na verdade, iremos utilizar apenas o *token* **\<eos\>** no nosso vocabulário, já que forneceremos a primeira letra do nome que queremos gerar para o nosso modelo. Ao utilizar o *token* **\<sos\>** no nosso contexto, a nossa rede terá que adivinhar às cegas o primeiro *token* do nosso nome. A utilização de ambos *tokens* de início e fim de sentença são mais vistos em abordagens *seq2seq*, onde produzimos um contexto da frase de entrada, auxiliando a rede a prever o *token* inicial a partir de **\<sos\>** com mais garantia de acerto.

In [21]:
vocabulary = []

# Adicionando o token especial <eos>
vocabulary.append('<eos>')

for names in language_names.values():
    for name in names:
        for token in name:        
            if token not in vocabulary:
                vocabulary.append(token)

print('Tamanho do alfabeto:', len(vocabulary))
print('5 tokens aleatórios do alfabeto:', random.choices(vocabulary, k=5))

Tamanho do alfabeto: 83
5 tokens aleatórios do alfabeto: ['-', 'u', 'u', 'ì', 'ê']


In [22]:
index2token = []
token2index = {}

for token_idx, token in enumerate(vocabulary):
    index2token.append(token)
    token2index[token] = token_idx

print('Token de índice 42:', index2token[42])
print('Índice do token "<eos>":', token2index['<eos>'])

Token de índice 42: p
Índice do token "<eos>": 0


Agora que temos as estruturas `index2token` e `token2index` bem definidas, conseguimos converter um nome em um tensor de índices numéricos através das funções `token2tensor` e `name2tensor`, como demonstrado a seguir.

> **Observação 1:** Criaremos um tensor de tamanho $n \times 1$, onde $n$ é o tamanho do nome e $1$ o tamanho do nosso *batch*. Para esse notebook, nós não iremos trabalhar com `batch_size > 1`, devido à complicações relacionadas com manipulação de sequências e *padding*. Deixaremos tais assuntos para serem abordados no notebook de `seq2seq`.

- Optamos por trocar a ordem padrão das dimensões dos dados em PyTorch, ou seja, com o tamanho do *batch* no começo, para facilitar operações em sequências no futuro, como utilizar `len` para obter o tamanho da sequência e tornar a indexação mais fácil. Na verdade, especificamente para sequências, PyTorch nos dá a opção de colocar o tamanho do *batch* como primeira dimensão do nosso tensor ou na segunda, sendo a primeira o tamanho da nossa sequência. Veremos isso em mais detalhes quando trabalharmos com os modelos recorrentes implementados pelo PyTorch.

In [24]:
def token2tensor(token):
    return torch.tensor([token2index[token]], dtype=torch.long)

def name2tensor(name):
    tensor = torch.zeros((1 + len(name), 1), dtype=torch.long)
    
    # Adicionando os tokens de fim de sentença
    tensor[-1] = token2tensor('<eos>')
    
    for idx, token in enumerate(name):
        tensor[idx] = token2tensor(token)

    return tensor

name = 'Jonas'
print(f'Tensor para o token J:', token2tensor('J'))
print(f'Tensor para o nome {name}: {name2tensor(name).T}')  # transposição para fins de print

Tensor para o token J: tensor([50])
Tensor para o nome Jonas: tensor([[50,  3, 25,  8,  9,  0]])


Para finalizar essa parte do notebook sobre dados, iremos criar uma função auxiliar `get_random_pair` para selecionar de forma aleatória um par `(idioma, nome)` da nossa base dados. Além disso, tal função irá retornar os tensores relacionados com cada componente do par.

- Por ora não iremos nos preocupar com a separação entre conjuntos de treino, validação e teste. O objetivo desse notebook é de ensinar como trabalhamos do zero com geração de sequências.

In [25]:
def language2tensor(language):
    return torch.tensor([languages.index(language)], dtype=torch.long)

def get_random_pair():
    language = random.choice(languages)
    name = random.choice(language_names[language])
    name = ''.join(name)  # convertendo lista de caracteres em string

    name_tensor = name2tensor(name)
    language_tensor = language2tensor(language)

    return name, language, name_tensor, language_tensor

In [26]:
name, language, name_tensor, language_tensor = get_random_pair()

print('Par selecionado:', (name, language))
print('Tensores do par:', (name_tensor.T, language_tensor))

Par selecionado: ('Torrens', 'English')
Tensores do par: (tensor([[24,  3,  5,  5, 11, 25,  9,  0]]), tensor([2]))


## Definição do nosso modelo

Como já tratamos a implementação de uma RNN do zero no notebook anterior, iremos utilizar o módulo `nn.RNN` disponibilizado pelo PyTorch. Uma documentação mais extensiva desse módulo pode ser encontrada [aqui](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#rnn). Não iremos utilizar todos os parâmetros customizáveis do módulo, mas iremos explicar eles aqui brevemente.

- **`input_size`**: o número de *features* da entrada, no nosso caso por exemplo o tamanho da dimensão de *embedding*.
- **`hidden_size`**: o tamanho do estado oculto ($h$) que utilizaremos.
- **`num_layers`**: número de camadas recorrentes que iremos "empilhar".
- **`nonlinearity`**: a camada de ativação não linear utilizado ao longo da recorrência.
- **`bias`**: valor *booleano* indicando se queremos ou não utilizar viés nas camadas.
- **`batch_first`**: valor *booleano* que permite a gente indicar se queremos trabalhar com dados onde a primeira dimensão é o tamanho do *batch*, nesse caso `batch_first=True`, ou o tamanho da sequência.
- **`dropout`**: introduz *dropout* em cada saída da camada recorrente (usado com `num_layers != 1`).
- **`bidirectional`**: valor *booleano* que indica se queremos um modelo recorrente bidirecional.

Logo abaixo temos um exemplo de utilização do módulo `nn.RNN`. Suponha que após aplicarmos o embedding, nós tenhamos a matriz `X`, cujas dimensões são: $S \times 1 \times E$, onde $S$ é o tamanho da sequência e $E$ o tamanho do embedding escolhido.

> Para esse exemplo utilizamos: $S = 4$, $E = 2$, $h = 8$.

In [15]:
seq_length = 4
embedding_size = 2
X = torch.rand(seq_length, 1, embedding_size)  # definindo uma matriz X aleatória apenas para exemplo

hidden_size = 8
model = nn.RNN(embedding_size, hidden_size, batch_first=False)

hidden = None  # nn.RNN irá selecionar h0 como sendo zeros
for Xt in X:
    outputs, hidden = model(Xt, hidden)

print('Tamanho do output final:', outputs.shape)
print('Tamanho do hidden final:', hidden.shape)

Tamanho do output final: torch.Size([1, 8])
Tamanho do hidden final: torch.Size([1, 8])


Como iremos trabalhar com geração de texto, **condicionando** a geração de nomes a partir de um idioma específico, a entrada do nosso modelo no tempo $t$ será a concatenação do *embedding* do *token* atual com o do idioma selecionado. Para isso, teremos dois *embeddings* diferentes: um para a transformação do idioma; e outro para transformação do *token*.

Um diagrama do esquema de geração pode ser visto a seguir, note que a nossa arquitetura possui uma modelagem autorregressiva, ou seja, a predição do tempo o $t$ depende das predições do tempo $t-1$, e assim por diante. Nele, o símbolo `&` representa o operador de concatenação das entradas.

> A geração da sentença será finalizada quando o modelo prever um *token* de fim de sentença ou quando atingirmos um tamanho predeterminado, forçando assim a adição de um `<eos>` no final da predição.

![](../imagens/rnn_autorregressivo.png)

In [16]:
class NameGenerationModel(nn.Module):
    def __init__(self, vocab_size, num_languages, embedding_size, hidden_size):
        super().__init__()

        self.letter_embedding = nn.Embedding(vocab_size, embedding_size)
        self.language_embedding = nn.Embedding(num_languages, embedding_size)

        self.rnn = nn.RNN(2 * embedding_size, hidden_size, batch_first=False)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, vocab_size),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, x, language, hidden = None):
        x = self.letter_embedding(x)
        language = self.language_embedding(language)
        combined = torch.cat([x, language], dim=-1)  # iremos concatenar ao longo das features

        output, hidden = self.rnn(combined, hidden)
        output = self.classifier(output)

        return output, hidden

Exemplo da utilização da rede para o tempo $t = 0$.

In [17]:
token = '<sos>'
language = 'Portuguese'

token_tensor = token2tensor(token)
language_tensor = language2tensor(language)

model = NameGenerationModel(len(vocabulary), len(languages), embedding_size=16, hidden_size=32)
output, hidden = model(token_tensor, language_tensor)

print('Saída do modelo:', output)
print('\nEstado oculto:', hidden)

next_token = output.argmax()
print('\nPróximo token previsto:', index2token[next_token])

Saída do modelo: tensor([[-4.8061, -4.3098, -4.3283, -4.4460, -4.8979, -4.6560, -4.3308, -4.6303,
         -3.8544, -4.6284, -4.1091, -4.3597, -4.5995, -4.1687, -4.7208, -4.7020,
         -4.4648, -4.1311, -4.6926, -4.8525, -4.2456, -4.9046, -4.3075, -4.5524,
         -3.9136, -4.3583, -4.6333, -3.9953, -4.6038, -4.4732, -4.9516, -4.2999,
         -4.6537, -4.6251, -4.5576, -4.1666, -4.4304, -4.1446, -4.0455, -4.1321,
         -4.9120, -4.2889, -4.4654, -4.9966, -4.4777, -4.3767, -4.8496, -4.5319,
         -4.4120, -4.8735, -4.5910, -4.3968, -4.3801, -4.3937, -4.6852, -4.3528,
         -4.0991, -4.3277, -4.6708, -4.0686, -4.3730, -4.2455, -4.5019, -4.0963,
         -4.3959, -4.6232, -4.5748, -4.8507, -4.7975, -4.6344, -4.5530, -4.4877,
         -4.4590, -4.1904, -4.2463, -4.2500, -4.2321, -5.0425, -4.3649, -4.1137,
         -4.3435, -4.6682, -4.4198, -4.7954]], grad_fn=<LogSoftmaxBackward0>)

Estado oculto: tensor([[-0.0822, -0.2311, -0.3425,  0.5709,  0.1809,  0.5220, -0.2139, -0.3118

## Treinamento do modelo

Para o treinamento do modelo generativo, iremos introduzir um novo conceito conhecido como *teacher forcing*, uma estratégia de treinamento de geração de sequências para acelerar a convergência do modelo e garantir estabilidade ao longo do treinamento. A estratégia consiste em utilizar como entrada para o tempo $t$ o *token* real do tempo $t-1$ ao invés da predição do modelo.

À primeira vista, essa estratégia pode parecer "roubada". Porém, temos que lembrar que o predição do modelo no tempo $t$ é dada pelo *ground truth* do tempo $t-1$ e o estado oculto do modelo que foi atualizado utilizando os *ground truths* do tempo $1$ até $t-2$. Ou seja, o modelo ainda está aprendendo os padrões linguísticos da sentença que lhe foi apresentada, mesmo não usando as suas próprias predições. Uma analogia interessante que podemos fazer com *teacher forcing* é quando vamos fazer uma prova onde a questão **d)** depende da resposta correta da questão **c)** e assim por diante. Se errarmos a questão **a)**, estaremos correndo o risco de errar todas as questões seguintes, mesmo realizando os cálculos corretos. Porém, se no começo de cada questão nós começarmos com a resposta correta da anterior, conseguimos acertar algumas coisas.

> **Importante:** Durante a geração de sentenças fora do treino, utilizaremos as predições do modelo ao invés do *ground truth* da sentença, uma vez que tal informação não está disponível. Sendo assim, é esperado observar uma discrepância da performance do modelo. Isso é conhecido na literatura como *exposure bias*, e uma forma simples de mitigar esse efeito é de utilizar *teacher forcing* de maneira probabilística, reduzindo a probabilidade de uso a medida que o modelo prevê novos *tokens* da sentença. Outra estratégia interessante pode ser lida no seguinte [artigo](https://arxiv.org/pdf/2103.11603.pdf), onde os autores propõem fornecer para o modelo um conjunto de palavras similares à do passo $t-1$ ao invés de um único *ground truth*.

- Apenas para relembrar: Como o nosso modelo recorrente possui como última camada uma `LogSoftmax`, iremos utilizar a função de perda `nn.NLLLoss` (*negative log-likelihood*), que espera log-probabilidades como saída da rede. Um fato curioso é que `nn.CrossEntropyLoss = LogSoftmax + nn.NLLLoss`, segundo a própria [documentação](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) da `nn.CrossEntropyLoss`.

In [19]:
model = NameGenerationModel(len(vocabulary), len(languages), embedding_size=64, hidden_size=128)
model = model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

loss_interval = 1000  # intervalo para salvar a loss média
print_interval = 5000  # intervalo para exibir performance da rede
num_iterations = 100000

# Variáveis para manter o valor da loss ao longo das épocas
all_losses = []
average_loss = 0

Algo a se notar aqui é que ao invés de usarmos a última saída da rede para calcular a função de custo, como no caso da classificação de nomes, estaremos realizando uma predição a cada instante de tempo. Logo, precisaremos calcular a função de custo para cada predição.

Para lidar com isso, o `autograd` do PyTorch permite você apenas somar todos os valores das funções de custo em cada instante de tempo e depois chamar `.backward()` no fim.

> **Importante:** A verificação da qualidade da sentença gerada será utilizada através do [BLEU Score](https://cloud.google.com/translate/automl/docs/evaluate?hl=pt-br#bleu), uma métrica usada para avaliação automática do texto traduzido por máquina. Note que estaremos utilizando essa pontuação fora do domínio na qual ela foi proposta, porém, ainda podemos obter *insights* interessantes ao utilizar essa pontuação, principalmente para verificar o quão discrepante a nossa geração está do *corpus* fornecido, uma vez que o *BLEU Score* verifica a similaridade da sentença gerada e do *corpus* através de um modelo de *n-grama*.

In [20]:
for iter in tqdm(range(1, num_iterations + 1)):
    name, language, name_tensor, language_tensor = get_random_pair()
    
    name_tensor = name_tensor.to(device)
    language_tensor = language_tensor.to(device)

    loss = 0
    hidden = None
    predicted_tokens = []

    # Iremos iterar sobre todos os tokens até o penúltimo
    for idx in range(len(name_tensor) - 1):
        output, hidden = model.forward(name_tensor[idx], language_tensor, hidden)  # note que estamos sempre fazendo teacher forcing
        loss += criterion(output, name_tensor[idx+1])

        # Salvando o nome predito pelo modelo
        token = index2token[output.argmax()]
        if token != '<eos>':
            predicted_tokens.append(token)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    average_loss +=  loss.item() / (len(name_tensor) - 1)
    
    if iter % print_interval == 0:
        predicted_name = ''.join(predicted_tokens)  # convertendo de lista de caracteres para string
        print(f'Iter [{iter}/{num_iterations}] => loss: {loss:.5f}, ' \
              f'language: {language}, predicted name: {predicted_name}, ' \
              f'BLEU score: {sentence_bleu(language_names[language], predicted_tokens):.5f}')

    if iter % loss_interval == 0:
        all_losses.append(average_loss / loss_interval)
        average_loss = 0

  0%|          | 0/100000 [00:00<?, ?it/s]

/u/ej0f/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/u/ej0f/.local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Iter [5000/100000] => loss: 17.67373, language: Spanish, predicted name: Rrrrr, BLEU score: 0.00000
Iter [10000/100000] => loss: 15.71425, language: English, predicted name: Eiann, BLEU score: 0.00000


KeyboardInterrupt: 

In [ ]:
plt.title('Evolução da função de perda média ao longo das iterações')
plt.xlabel('Intervalo de Salvamento')
plt.ylabel('Valor médio')
    
plt.plot(all_losses)
plt.show()

## Avaliando os resultados obtidos

Para verificar o desempenho da rede em diferentes línguas, criaremos uma matriz de confusão, indicando para cada idioma real (linhas) qual idioma a rede adivinha (colunas). Para calcular a matriz de confusão, uma quantia grande de amostras, definida pela variável `num_confusion_samples`, serão processadas pela rede em modo de avaliação.

In [ ]:
num_confusion_samples = 10000
confusion_matrix = torch.zeros(len(languages), len(languages))

model.eval()
with torch.no_grad():
    for i in tqdm(range(num_confusion_samples)):
        name, language, name_tensor, language_tensor = get_random_pair()

        name_tensor = name_tensor.to(device)
        language_tensor = language_tensor.to(device)

        output = model(name_tensor)
        guess = get_language_from_output(output)

        guess_idx = languages.index(guess)
        language_idx = languages.index(language)

        confusion_matrix[language_idx, guess_idx] += 1

# Normalizando as linhas da matriz de confusão
for i in range(len(languages)):
    confusion_matrix[i] /= confusion_matrix[i].sum()

# Configurando o plot
fig, ax = plt.subplots()

img = ax.matshow(confusion_matrix.numpy())
fig.colorbar(img)

ax.set_xticks(range(len(languages)), languages, rotation=90)
ax.set_yticks(range(len(languages)), languages)

fig.tight_layout()

## Verificando as top 3 predições da rede

In [ ]:
def predict(name):
    print(f'\n> {name}')

    model.eval()
    with torch.no_grad():
        name_tensor = name2tensor(name).to(device)
        output = model(name_tensor)

        vals, idxs = torch.topk(output[0], k=3, dim=-1)
        for val, idx in zip(vals, idxs):
            prob = torch.e ** val  # probabilidade = exp(log-probabilidade)
            print(f'({prob:.2f}) {languages[idx]}')
            
predict('Dovesky')
predict('Jackson')
predict('Satoshi')